In [16]:
import re
import os

In [17]:
# Read file

In [18]:
with open("Standard2.svg") as f:
    filecontent = f.read().split("<defs/>\n ")

In [19]:
len(filecontent)

In [20]:
# Districts not to be part in the game
blocked_stroek = [7, 38]

In [21]:
# Find numbers and sort SVG lines

In [22]:
svgDict = {}

In [23]:
for idx, single_svg in enumerate(filecontent):
    if idx != 0 and idx not in blocked_stroek:
        match = re.search(r'fid_(\d+)', single_svg)
        
        lines = single_svg.split("\n")
        newLines = []
        for idx, line in enumerate(lines):
            if line.strip().startswith("<g inkscape:groupmode=\"layer\""):
                newLines.append(line)
            elif line.strip().startswith("<g fill-opacity=\"1\""):
                l = re.sub(r'transform="[^"]*"', '', line)
                newLines.append(l)
            elif line.strip().startswith("<path d="):
                newLines.append(line)
        newLines.append("\n</g>\n</g>")
                
        
        svgDict[int(match.group(1))] = "\n".join(newLines)

In [24]:
extremePoints = {}

In [25]:
def findExtremePoints(pathstr):
    arr = pathstr.split()
    min_x = float("inf")
    min_y = float("inf")
    max_x = float("-inf")
    max_y = float("-inf")
    for a in arr:
        x,y = a.replace("L","").replace("M","").split(",")
        x = float(x)
        y = float(y)
        if x > max_x:
            max_x = x
        if y > max_y:
            max_y = y
        if x < min_x:
            min_x = x
        if y < min_y:
            min_y = y
    #return f"{round(float(min_x+20), 3)} {round(float(min_y+20), 3)} {round(float(max_x-min_x+40), 3)} {round(float(max_y-min_y+40), 3)}"
    return f"{round(float(min_x-5), 3)} {round(float(min_y-5), 3)} {round(float(max_x-min_x+10), 3)} {round(float(max_y-min_y+10), 3)}"
    

In [26]:
# Change size of shape

In [27]:
for idx, line in svgDict.items():
    if idx != 0 and idx not in blocked_stroek:
        #cont = svgDict[idx].split("\n")
        #match = re.search(r'path', line)
        pattern = r'<path[^>]*\s+d="([^"]+)"'
        
        matches = re.findall(pattern, line)
        
        extremePoints[idx] = findExtremePoints(matches[0])

In [31]:
# Write SVGs

In [32]:
for idx in range(1,len(filecontent)):
    if idx not in blocked_stroek:
        filename = f"../public/images/countries/{idx}/vector.svg"
        #os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(f"{filename}","w") as toWrite:
            toWrite.write(f"<svg xmlns=\"http://www.w3.org/2000/svg\" version=\"1.2\" baseProfile=\"tiny\" xmlns:inkscape=\"http://www.inkscape.org/namespaces/inkscape\" width=\"315.045mm\" viewBox=\"{extremePoints[idx]}\" height=\"227.076mm\" xmlns:xlink=\"http://www.w3.org/1999/xlink\" xmlns:cc=\"http://creativecommons.org/ns#\">\n")
            toWrite.write(f"{svgDict[idx]}\n")
            toWrite.write("</svg>")